<h1 style="text-align:center">BERT для решения задачи multi-label классификации текстов.</h1>

<center> <h2>1. Импорт данных и приведение таргета к нужному виду. </h2> </center>

In [1]:
import numpy as np
import pandas as pd

In [2]:
X_train_df = pd.read_csv("../data/processed/train/text_train_df.csv", index_col=0)
X_val_df = pd.read_csv("../data/processed/val/text_val_df.csv", index_col=0)

y_train_df = pd.read_csv("../data/processed/train/target_train_df.csv", index_col=0)
y_val_df = pd.read_csv("../data/processed/val/target_val_df.csv", index_col=0)

In [3]:
# Заполним пропуски, которые возникли из-за наличия отзывов без текста
X_train_df = X_train_df.fillna('')
X_val_df = X_val_df.fillna('')

In [4]:
target_cols = [f"trend_id_res{i}" for i in range(50)]

In [5]:
def make_target_list(row) -> list:
    target_list = []
    for target in target_cols:
        target_list.append(row[target])
    return target_list

In [6]:
y_train_df = pd.DataFrame(y_train_df.apply(make_target_list, axis=1), columns=['target_list'])
y_val_df = pd.DataFrame(y_val_df.apply(make_target_list, axis=1), columns=['target_list'])
display(y_train_df.head())
display(y_val_df.head())

,target_list
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


,target_list
4161,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4162,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
4163,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4164,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4165,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


<center><h2> 2. Приведение данных к torch формату для скармливанию модельке.</h2><center>

In [7]:
import torch
from torch.utils.data import DataLoader, Dataset

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
class TextDataset(Dataset):
    """Класс для преобразования датасета к нужному формату"""
    def __init__(self, X, y, tokenizer):
        self.tokenizer = tokenizer
        self.sentences = X['text'].tolist()
        self.labels = y['target_list'].tolist()

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        text = self.sentences[idx]
        target_list = self.labels[idx]
        # токенизируем
        inputs = self.tokenizer.encode_plus(
            text=text, 
            add_special_tokens=True, # добавление спец-токенов, отвечающих за "начало предложения" [CLS] и "конец предложения" [SEP]
            padding='max_length', 
            truncation=True, 
            return_token_type_ids=False, # это для задачи вопросно-ответной системы, т.е. не для нас
            return_attention_mask=True, 
            return_tensors='pt' # формат выдачи токенизатора, в нашем случае - torch тензор
        )

        # то что мы запихнем в модель
        return {
            'input_ids': inputs['input_ids'].flatten(), # это наши цифровые токены (т.е. для токена 'привет' будет какое-нибудь '105')
            'attention_mask': inputs['attention_mask'].flatten(), # это наши маски
            'labels': torch.tensor(target_list, dtype=torch.float)
        }

In [10]:
from transformers import AutoTokenizer, AutoModel

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
# Маленький берт, легче, быстрее.
rubert_tiny_model = AutoModel.from_pretrained("cointegrated/rubert-tiny", return_dict=False)
rubert_tiny_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")

# Берт покрупнее
rubert_model = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased', return_dict=False)
rubert_tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
train_dataset = TextDataset(X_train_df, y_train_df, rubert_tiny_tokenizer)
val_dataset = TextDataset(X_val_df, y_val_df, rubert_tiny_tokenizer)

In [14]:
batch_size = 16

train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

<center> <h2>3. Создадим класс нейронки для решения задачи</h2> </center>

In [15]:
class BertMultiLabel(torch.nn.Module):
    def __init__(self, bert, n_classes):
        super(BertMultiLabel, self).__init__()
        self.bert = bert
        self.dropout = torch.nn.Dropout(0.2)
        self.fc = torch.nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, ids, mask):
        _, output = self.bert(ids, attention_mask=mask)
        output = self.dropout(output)
        output = self.fc(output)
        return output

In [16]:
custom_model = BertMultiLabel(rubert_tiny_model, 50)
custom_model.to(device)

BertMultiLabel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29564, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_

In [17]:
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params =  custom_model.parameters(), lr=1e-05)

<center><h2>4. Обучение модели</h2></center>

In [18]:
import tqdm
from tqdm.auto import tqdm

def train_val(model, train_data_loader, val_data_loader, loss_fn, optimizer, device, num_epochs, save_path="../models/tiny_rubert"):
    for t in tqdm(range(num_epochs)):
        train_epoch_loss = []
        eval_epoch_loss = []
        
        print("Эпоха номер: ", t)

        # Обучение модели на текущей эпохе
        print("Обучение началось...")
        batch_counter = 0
        all_batches_count = len(train_data_loader)
        
        model.train()
        for train_data in tqdm(train_data_loader):
            
            batch_counter += 1
            if batch_counter % 50 == 0:
                print(f"Прошло {batch_counter} батчей из {all_batches_count}")
            
            input_ids = train_data['input_ids'].to(device) # токены
            attention_mask = train_data['attention_mask'].to(device) # маски
            labels = train_data['labels'].to(device) # класс

            outputs = model(input_ids, attention_mask) # результат модели
            #_, preds = torch.max(outputs.logits, dim=1)
            
            loss = loss_fn(outputs, labels) # считаем потерю
            train_epoch_loss.append(loss.item())


            # Выполним подсчёт новых градиентов
            loss.backward()
            # Выполним шаг градиентного спуска
            optimizer.step()
            # Обнулим сохраненные у оптимизатора значения градиентов
            # перед следующим шагом обучения
            optimizer.zero_grad()

        # Оценка модели на валидационных данных после обучения на текущей эпохе
        print("Оцениваем модель после эпохи обучения...")
        model.eval()
        for val_data in tqdm(val_data_loader):
            input_ids = val_data['input_ids'].to(device) # токены
            attention_mask = val_data['attention_mask'].to(device) # маски
            labels = val_data['labels'].to(device) # класс


            with torch.no_grad():
                outputs = model(input_ids, attention_mask) # результат модели
            
                loss = loss_fn(outputs, labels) # считаем потерю
                eval_epoch_loss.append(loss.item())

        # Выведем результаты прошедшей эпохи обучения
        print("Train loss: ", np.mean(train_epoch_loss))
        print("Eval loss: ", np.mean(eval_epoch_loss))
        
        torch.save(model.state_dict(), save_path)
    return model

In [19]:
trained_custom_model = train_val(custom_model, train_data_loader, val_data_loader, loss_fn, optimizer, device, 3)

  0%|          | 0/3 [00:00<?, ?it/s]

Эпоха номер:  0
Обучение началось...


  0%|          | 0/261 [00:00<?, ?it/s]

Прошло 50 батчей из 261
Прошло 100 батчей из 261
Прошло 150 батчей из 261
Прошло 200 батчей из 261
Прошло 250 батчей из 261
Оцениваем модель после эпохи обучения...


  0%|          | 0/29 [00:00<?, ?it/s]

...

Train loss:  0.22822394584558933
Eval loss:  0.17745777450758835
Эпоха номер:  2
Обучение началось...


  0%|          | 0/261 [00:00<?, ?it/s]

Прошло 50 батчей из 261
Прошло 100 батчей из 261
Прошло 150 батчей из 261
Прошло 200 батчей из 261
Прошло 250 батчей из 261
Оцениваем модель после эпохи обучения...


  0%|          | 0/29 [00:00<?, ?it/s]

Train loss:  0.16076384753103
Eval loss:  0.14217947012391582


In [20]:
torch.save(trained_custom_model.state_dict(), "../models/tiny_rubert")

<center><h2>5. Считаем accuracy на валидационной части.</h2></center>

In [21]:
from sklearn.metrics import accuracy_score

In [22]:
def eval_accuracy(model, X_val, y_val, tokenizer):
    model.eval()
    with torch.no_grad():
        text_dataset = TextDataset(X_val, y_val, tokenizer)
        data_loader = DataLoader(text_dataset, batch_size=len(text_dataset), shuffle=False)
        data = next(iter(data_loader))

        input_ids = data['input_ids'].to(device) # токены
        attention_mask = data['attention_mask'].to(device) # маски
        targets = data['labels']

        output = model(input_ids, attention_mask)

        predictions = (output.detach().numpy() >= 0.5).astype(int)
        y_true = targets.detach().numpy().astype(int)

    return accuracy_score(y_true, predictions)  

In [25]:
eval_accuracy(custom_model, X_train_df, y_train_df, rubert_tiny_tokenizer)

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 2658779136 bytes.

In [24]:
print(len(train_data_loader))

261
